In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import keras.applications.inception_v3 as inception

N_CLASSES = 2
IMSIZE = (299, 299)

# TO DO:: Replace these with paths to the downloaded data.
# Training directory
train_dir = '/mnt/e/data/catdog/train'
# Testing directory
test_dir = '/mnt/e/data/catdog/validation'

train_dir = r'E:\workshare\Mind\A3\data\catdog\train'
test_dir = r"E:\workshare\Mind\A3\data\catdog\validation"
# Start with an Inception V3 model, not including the final softmax layer.
base_model = inception.InceptionV3(weights='imagenet')
print ('Loaded Inception model')

Using TensorFlow backend.


Loaded Inception model


In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
for layer in base_model.layers:
    print(layer.output)

Tensor("input_1:0", shape=(?, 299, 299, 3), dtype=float32)
Tensor("conv2d_1/convolution:0", shape=(?, 149, 149, 32), dtype=float32)
Tensor("batch_normalization_1/cond/Merge:0", shape=(?, 149, 149, 32), dtype=float32)
Tensor("activation_1/Relu:0", shape=(?, 149, 149, 32), dtype=float32)
Tensor("conv2d_2/convolution:0", shape=(?, 147, 147, 32), dtype=float32)
Tensor("batch_normalization_2/cond/Merge:0", shape=(?, 147, 147, 32), dtype=float32)
Tensor("activation_2/Relu:0", shape=(?, 147, 147, 32), dtype=float32)
Tensor("conv2d_3/convolution:0", shape=(?, 147, 147, 64), dtype=float32)
Tensor("batch_normalization_3/cond/Merge:0", shape=(?, 147, 147, 64), dtype=float32)
Tensor("activation_3/Relu:0", shape=(?, 147, 147, 64), dtype=float32)
Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 73, 73, 64), dtype=float32)
Tensor("conv2d_4/convolution:0", shape=(?, 73, 73, 80), dtype=float32)
Tensor("batch_normalization_4/cond/Merge:0", shape=(?, 73, 73, 80), dtype=float32)
Tensor("activation_4/Relu:0",

In [3]:
for layer in base_model.layers:
    print(layer.name)

input_1
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
conv2d_9
batch_normalization_9
activation_9
conv2d_7
conv2d_10
batch_normalization_7
batch_normalization_10
activation_7
activation_10
average_pooling2d_1
conv2d_6
conv2d_8
conv2d_11
conv2d_12
batch_normalization_6
batch_normalization_8
batch_normalization_11
batch_normalization_12
activation_6
activation_8
activation_11
activation_12
mixed0
conv2d_16
batch_normalization_16
activation_16
conv2d_14
conv2d_17
batch_normalization_14
batch_normalization_17
activation_14
activation_17
average_pooling2d_2
conv2d_13
conv2d_15
conv2d_18
conv2d_19
batch_normalization_13
batch_normalization_15
batch_normalization_18
batch_normalization_19
activation_13
activation_15
activation_18
activation_19
mixed1
conv2d_23
batch_normalization_23
activat

In [5]:
# Turn off training on base model layers
for layer in base_model.layers:
    layer.trainable = True
# k=base_model.get_layer('flatten').output
# Add on new fully connected layers for the output classes.
x = Dense(32, activation='relu')(base_model.get_layer('avg_pool').output)
x = Dropout(0.5)(x)
predictions = Dense(N_CLASSES, activation='softmax', name='predictions')(x)

model = Model(inputs=base_model.input, outputs=[predictions])

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


# Show some debug output
print (model.summary())

print ('Trainable weights')
print (model.trainable_weights)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 149, 149, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [7]:
# Data generators for feeding training/testing images to the model.
model.load_weights('catdog_pretrain.h5') 
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        steps_per_epoch = 5,
        epochs = 5,
        validation_data = test_generator,
        verbose = 2,
        validation_steps = 1)
model.save_weights('catdog_pretrain.h5')  # always save your weights after training or during training




# img_path = '../data/sport3/validation/hockey/img_2997.jpg'
# img = image.load_img(img_path, target_size=IMSIZE)
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)

# x = inception.preprocess_input(x)

# preds = model.predict(x)
# print('Predicted:', preds)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/5
5s - loss: 0.1886 - acc: 0.9375 - val_loss: 0.0826 - val_acc: 0.9688
Epoch 2/5
5s - loss: 0.2309 - acc: 0.8937 - val_loss: 0.0888 - val_acc: 0.9688
Epoch 3/5
5s - loss: 0.2073 - acc: 0.9437 - val_loss: 0.1387 - val_acc: 0.9375
Epoch 4/5
5s - loss: 0.1345 - acc: 0.9500 - val_loss: 0.0397 - val_acc: 1.0000
Epoch 5/5
5s - loss: 0.0970 - acc: 0.9813 - val_loss: 0.0983 - val_acc: 0.9688


In [ ]:
model.load_weights('catdog_pretrain.h5') 
img_path = '../data/dog1.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)